# Lab 1: Information Retrieval

__Students:__ 

### Crawling



a) Get the webpage content by using functions in 
__[urllib module](https://docs.python.org/3/library/urllib.html#module-urllib)__.

Other libraries are also fine to achieve the crawling.

e.g. scrapy, beautifulsoup... 

In [59]:
import urllib.request
import re


x = urllib.request.urlopen('https://play.google.com/store/apps/category/GAME?hl=en').read().decode('utf-8')
#print(x)



In [60]:
appreg = r'href=\"(/store/apps/details.*?)\"'
appre = re.compile(appreg)
app_url_list = re.findall(appre,x)


reg1 = r'href=\"(/store/apps/category/GAME_.*?)\"'
cat_re = re.compile(reg1)
cat_url_list = re.findall(cat_re,x)
print(cat_url_list)

['/store/apps/category/GAME_ACTION', '/store/apps/category/GAME_ADVENTURE', '/store/apps/category/GAME_ARCADE', '/store/apps/category/GAME_BOARD', '/store/apps/category/GAME_CARD', '/store/apps/category/GAME_CASINO', '/store/apps/category/GAME_CASUAL', '/store/apps/category/GAME_EDUCATIONAL', '/store/apps/category/GAME_MUSIC', '/store/apps/category/GAME_PUZZLE', '/store/apps/category/GAME_RACING', '/store/apps/category/GAME_ROLE_PLAYING', '/store/apps/category/GAME_SIMULATION', '/store/apps/category/GAME_SPORTS', '/store/apps/category/GAME_STRATEGY', '/store/apps/category/GAME_TRIVIA', '/store/apps/category/GAME_WORD', '/store/apps/category/GAME_ACTION', '/store/apps/category/GAME_ADVENTURE', '/store/apps/category/GAME_ARCADE', '/store/apps/category/GAME_BOARD', '/store/apps/category/GAME_CARD', '/store/apps/category/GAME_CASINO', '/store/apps/category/GAME_CASUAL', '/store/apps/category/GAME_EDUCATIONAL', '/store/apps/category/GAME_MUSIC', '/store/apps/category/GAME_PUZZLE', '/store/a

b) Get app url by regular expression using functions from __[re module](https://docs.python.org/3/library/re.html?highlight=re#module-re)__.

A useful online regular expression check.
__[Check your regular expression first](https://regex101.com)__.

In [61]:
start = 'https://play.google.com'
url_ending = '/collection/topselling_new_free?hl=en'
url_ending2 = '/collection/topgrossing?hl=en'
for e in cat_url_list:
    x = urllib.request.urlopen(start+e+url_ending).read().decode('utf-8')
    app_url_list += re.findall(appre,x)
    x = urllib.request.urlopen(start+e+url_ending2).read().decode('utf-8')
    app_url_list += re.findall(appre,x)
app_url_list = list(set(app_url_list))
print(len(app_url_list))
print(app_url_list[:10])

1699
['/store/apps/details?id=com.highway.motorrider', '/store/apps/details?id=com.ninjakiwi.bloonstd6', '/store/apps/details?id=com.gsn.android.tripeaks', '/store/apps/details?id=com.Heyworks.PocketTroopsVK', '/store/apps/details?id=com.warducks.smooshymushy', '/store/apps/details?id=com.dmi.nascarheat', '/store/apps/details?id=com.NinJaGo.Shadow', '/store/apps/details?id=com.tndh.cutepuppycare', '/store/apps/details?id=com.cg.real.car.hill.racing.games', '/store/apps/details?id=mobi.square.sr.android']


c) Access specific webpage to get description of each app and then store the description in files.

In [43]:

desc_re = r'itemprop=\"description.*?\">.*?<div jsname=\".*?\">.*?</div>'
desc_reg = re.compile(desc_re)
descriptions = []

for url_ in app_url_list:
    x = urllib.request.urlopen(start+url_+'&hl=en').read().decode('utf-8')
    descriptions += re.findall(desc_reg,x)
print(descriptions[4])

itemprop="description"><content>What’s New:<br>♠ Coming soon: a whole NEW WORLD! Come explore Tiki Paradise and test your skills in a new kind of gameplay. Hint: two cards are better than one!<br>♠ Bug fixes: As always, if you spot a glitch, drop us a line and we’ll look into it.</content><div jsname="WgKync" class="uwAgLc"></div>


In [53]:
get_text_re = r'(?<=\<content\>)(.*)(?=\</content)'
get_text_reg = re.compile(get_text_re)
desc_texts = []
for desc in descriptions:
    tmp = re.findall(get_text_re,desc)
    if tmp != []:
        text = tmp[0].replace('<br>',' ')
        desc_texts += [text]
        
print(desc_texts[22])

EXCLUSIVE RELEASES: 918 Spyder MARTINI RACING, Cayman GT4 ‘Salzburg’ In honour of Porsche’s iconic history, CSR2 brings you the Cayman GT4 ‘Salzburg’ and the 918 Spyder MARTINI RACING.   NEW EVENTS: Porsche 70th Anniversary Finale, Halloween CSR2 are bringing you a finale to remember for Porsche&#39;s 70th anniversary and a spooky Halloween treat.  NEW CARS: Porsche, Mercedes, Chevrolet Four new cars available for the first time in a game and eight other ones for your collection!


In [54]:
import numpy as np

np.save('descriptions',desc_texts)

In [71]:
desc_texts = np.load('descriptions.npy')

### Construct Inverted file index (Vector Model)



d) Preprocess text using NLP techniques from __[nltk module](http://www.nltk.org/py-modindex.html)__ or spaCy.

Using nltk.download(ID) to get the corpora if it is not downloaded before. __[nltk corpora](http://www.nltk.org/nltk_data/)__

In [44]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/johmy592/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [81]:
import string

for i in range(len(desc_texts)):
    desc_texts[i] = ''.join(ch.lower() for ch in desc_texts[i] if ch.isalnum() or ch==' ')
    desc_texts[i] = desc_texts[i].replace('39','\'')
    
print(desc_texts[22])

exclusive releases 918 spyder martini racing cayman gt4 salzburg in honour of porsches iconic history csr2 brings you the cayman gt4 salzburg and the 918 spyder martini racing   new events porsche 70th anniversary finale halloween csr2 are bringing you a finale to remember for porsches 70th anniversary and a spooky halloween treat  new cars porsche mercedes chevrolet four new cars available for the first time in a game and eight other ones for your collection


In [89]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

tokens = [word_tokenize(t) for t in desc_texts]
tokens = [[wnl.lemmatize(t) for t in d if t not in stopwords.words('english')] for d in tokens]
print(tokens[6])

['update', 'includes', 'improvement', 'provide', 'better', 'gaming', 'experience', 'constantly', 'strive', 'make', 'game', 'better', 'feedback', 'always', 'welcome', 'thank', 'playing', 'archery', 'king']


...)Compute tdidf 
eg. Using functions from __[scikit-learn module](http://scikit-learn.org/stable/modules/classes.html)__. TfidfVectorizer is used for converting a collection of raw documents to a matrix of TF-IDF features.
#### You can also build the tfidf matrix with other library or your own algorithm

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer
transvector = TfidfVectorizer()
corpus = [
     'This is the first document.',
     'This is the second second document.',
     'And this is the third one.',
     'Is this the first document?',]
tfidf1 = transvector.fit_transform(corpus)
print(tfidf1.toarray())
print(transvector.get_feature_names())


[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.27925389 0.         0.22830836 0.         0.87501037
  0.22830836 0.         0.22830836]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [105]:
transvector = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x)
tfidf1 = transvector.fit_transform(tokens)
print(tfidf1)
#print(tfidf1.toarray())
#print(transvector.get_feature_names())

  (0, 4836)	0.16491941725871823
  (0, 3218)	0.20604277934852913
  (0, 1836)	0.14753068686829068
  (0, 3135)	0.18709126624900868
  (0, 548)	0.18709126624900868
  (0, 4177)	0.16491941725871823
  (0, 1280)	0.1311047761741568
  (0, 1607)	0.18099024174164716
  (0, 3548)	0.1949568548533839
  (0, 1959)	0.31363936703741074
  (0, 4624)	0.17600534175386343
  (0, 2660)	0.20604277934852913
  (0, 1178)	0.20604277934852913
  (0, 4311)	0.18099024174164716
  (0, 2668)	0.1949568548533839
  (0, 795)	0.1949568548533839
  (0, 2319)	0.20604277934852913
  (0, 3517)	0.18709126624900868
  (0, 3118)	0.20604277934852913
  (0, 4315)	0.20604277934852913
  (0, 3730)	0.20604277934852913
  (0, 1957)	0.12290985484435454
  (0, 4921)	0.14753068686829068
  (0, 3538)	0.20604277934852913
  (0, 2617)	0.10768775123670149
  :	:
  (1377, 431)	0.11422626953866727
  (1377, 2172)	0.13409704851216245
  (1377, 4266)	0.1521251228230917
  (1377, 3663)	0.12130592622723502
  (1377, 3577)	0.12592202983669498
  (1377, 4221)	0.2862221713

In [122]:
import scipy as sp
from scipy.sparse import csr_matrix

array([1., 1., 1., ..., 1., 1., 1.])

<1x5563 sparse matrix of type '<class 'numpy.float64'>'
	with 28 stored elements in Compressed Sparse Row format>

### Query Process

eg. "Dragon, Control, hero, running"

eg. "The hero controls the dragon to run."



In [127]:
test_search = "shoot guns ¥{[]}"

def preprocess_search(search_term):
    tolower = ''.join(ch.lower() for ch in search_term if ch.isalnum() or ch==' ')
    tok = word_tokenize(tolower) 
    return [wnl.lemmatize(t) for t in tok if t not in stopwords.words('english')]
    
real_search = preprocess_search(test_search)
search_vec = transvector.transform([real_search])
print(search_vec)

similarities = tfidf1.dot(search_vec.transpose())
print(np.argmax(similarities))
print(np.amax(similarities))

  (0, 4305)	0.7328705696650658
  (0, 2244)	0.6803680828190003
287
0.41638954255743044


In [164]:
def get_top_results(search_term,k):
    real_search = preprocess_search(search_term)
    search_vec = transvector.transform([real_search])
    similarities = tfidf1.dot(search_vec.transpose())
    print(np.argmax(similarities.todense()))
    print(np.shape(similarities.toarray()))
    print(np.shape(similarities.toarray()[:,0]))
    return np.argsort(similarities.toarray()[:,0])[-k:][::-1]
     
results = get_top_results('poker omaha',5)
print(results)
for i in results:
    print(desc_texts[i])
    print('############################################','\n')

1119
(1378, 1)
(1378,)
[1119  726  800  166  685]
the all new video poker app is here to keep your video poker itch scratched for hours get up to 8x the winnings in multistrike poker play up to 25 hands at once in multiplay poker or stick to the tried and true classics if you love video poker youll love pechangas best bet video poker
############################################ 

jump in for the best poker experience ever mega hit poker is rng ready certified by itech labs  whats new  bug fixes amp game optimization
############################################ 

the new world poker club version is out weve tuned up the speed and stability of the app as well as added some gifts and updated the special promos which now offer even more deals and bargains dont forget to check out the new events too the clatter of chips the clinking of coins and the rustle of cards are all music to the ears of any devout poker fan  cant wait for you to join us at the tables place your bets ladies and gentle